# Download and Import Dependencies
Joint task

In [ ]:
# pip install -r requirements.txt

In [ ]:
!pip install pandasql

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandasql import sqldf

# Data Retrieval
Joint task


In [ ]:
from google.colab import drive
drive.mount('/drive')
%cd ..
%cd /drive/MyDrive/DIC_PROJECT
!ls

# comment this if you working locally. Assuming all the csv files are here locally where this ipynb file is located.
# csv files won't be committed and won't be seen in the changes as .csv files are in ignored.

In [ ]:
Athlete_Events_Details = pd.read_csv('Olympic_Athlete_Event_Details.csv')
Event_Results = pd.read_csv('Olympic_Event_Results.csv')
Athlete_Biography = pd.read_csv('Olympic_Athlete_Biography.csv')
Medal_Tally = pd.read_csv('Olympic_Medal_Tally_History.csv')
Games_Summary = pd.read_csv('Olympic_Games_Summary.csv')
Population_Total = pd.read_csv('population_total_long.csv')
Country_Profile = pd.read_csv('Olympic_Country_Profiles.csv')

In [ ]:
print(Athlete_Events_Details.shape)
print(Event_Results.shape)
print(Athlete_Biography.shape)
print(Medal_Tally.shape)
print(Games_Summary.shape)
print(Population_Total.shape)

# Event Results Cleaning
Cleaned by Geetansh


In [ ]:
print(Event_Results.shape)
print(Event_Results.describe())
# print(Event_Results.head(10))

1. Removing duplicates if any.

In [ ]:
Event_Results.drop_duplicates(inplace=True)

2. Removing unwanted columns.


In [ ]:
Event_Results.drop(['sport_url','result_date','result_location','result_format','result_detail','result_description'], axis=1, inplace=True)

In [ ]:
print(Event_Results.shape)

3. Will convert all text fields to lower for consistency


In [ ]:
Event_Results['event_title'] = Event_Results['event_title'].str.strip().str.lower()
Event_Results['edition'] = Event_Results['edition'].str.strip().str.lower()
Event_Results['sport'] = Event_Results['sport'].str.strip().str.lower()
Event_Results['result_participants'] = Event_Results['result_participants'].str.strip().str.lower()

In [ ]:
print(Event_Results['result_participants'].head(10))

4. Creating new columns with total participants and total participating countries for each event

In [ ]:
Event_Results[['participants', 'participant_countries']] = Event_Results['result_participants'].str.extract(r'(\d+)\sfrom\s(\d+)')

Event_Results['participants'] = Event_Results['participants'].astype(int)
Event_Results['participant_countries'] = Event_Results['participant_countries'].astype(int)

Event_Results.drop('result_participants', axis=1, inplace=True)

print(Event_Results)

In [ ]:
print(Event_Results['participants'].head(10))
print(Event_Results['participant_countries'].head(10))

In [ ]:
print(Event_Results['event_title'].head(10))

5. Separating male and female events by adding a column for each men and women

In [ ]:
Event_Results['men'] = Event_Results['event_title'].str.contains(r'\bmen\b', case=False).astype(int)
Event_Results['women'] = Event_Results['event_title'].str.contains('women', case=False).astype(int)

print(Event_Results)

6. Extract year and olympic type from the data

In [ ]:
Event_Results[['year', 'olympic_type']] = Event_Results['edition'].str.extract(r'(\d{4})\s+(summer|winter)\s+olympics')

print(Event_Results['year'].head())
print(Event_Results['olympic_type'].head())

In [ ]:
print(Event_Results.shape)

7. Drop rows where essential columns are null

In [ ]:
Event_Results.dropna(subset=['event_title', 'sport', 'participants', 'participant_countries', 'men', 'women','year','olympic_type'], inplace=True)

In [ ]:
print(Event_Results)

8. One hot encode olympic type

In [ ]:
olympic_dummies = pd.get_dummies(Event_Results['olympic_type'])

olympic_dummies = olympic_dummies.astype(int)

Event_Results = pd.concat([Event_Results, olympic_dummies], axis=1)

print(Event_Results)

In [ ]:
print(Event_Results.shape)
print(Event_Results.describe())
print(Event_Results.head(10))

# Population Total Cleaning
Cleaned by Geetansh

In [ ]:
print(Population_Total.shape)
print(Population_Total.describe())
print(Population_Total.head(10))

1. Text columns to lower text

In [ ]:
Population_Total['Country Name'] = Population_Total['Country Name'].str.strip().str.lower()

In [ ]:
print(Population_Total.head(10))

2. Normalize Population Count

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()

# Population_Total[['Count']] = scaler.fit_transform(Population_Total[['Count']])

# print(Population_Total)

3. Drop rows if essential columns are null

In [ ]:
Population_Total.dropna(subset=['Country Name', 'Year', 'Count'], inplace=True)

In [ ]:
Population_Total['Year'] = Population_Total['Year'].replace(2017, 2020)

In [ ]:
print(Population_Total.shape)
print(Population_Total.describe())
print(Population_Total)

# Athlete_Biography Cleaning
Cleaned Sataakshi

In [ ]:
print(Athlete_Biography.shape)
print(Athlete_Biography.describe())
print(Athlete_Biography.head(10))

1. Dropping duplicates

In [ ]:
Athlete_Biography.drop_duplicates(inplace=True)

2. Dropping non required columns

In [ ]:
Athlete_Biography.drop(['description','special_notes'], axis=1, inplace=True)

3. Converting strings to lower

In [ ]:
Athlete_Biography['country'] = Athlete_Biography['country'].str.strip().str.lower()
Athlete_Biography['sex'] = Athlete_Biography['sex'].str.strip().str.lower()
Athlete_Biography['name'] = Athlete_Biography['name'].str.strip().str.lower()
Athlete_Biography['born'] = Athlete_Biography['born'].str.strip().str.lower()

In [ ]:
print(Athlete_Biography.head(10))

# Data Cleaning: Athlete_Events
Cleaned by Sataakshi

In [ ]:
Athlete_Events_Details.head(10)

In [ ]:
print(Athlete_Events_Details.isnull().sum())

In [ ]:
Athlete_Events_Details = Athlete_Events_Details.drop_duplicates()

In [ ]:
Athlete_Events_Details = Athlete_Events_Details.drop(columns=['result_id', 'athlete', 'pos'])

In [ ]:
Athlete_Events_Details['medal'].fillna('no medal', inplace=True)

In [ ]:
Athlete_Events_Details['country_noc'] = Athlete_Events_Details['country_noc'].str.strip().str.lower()
Athlete_Events_Details['sport'] = Athlete_Events_Details['sport'].str.strip().str.lower()
Athlete_Events_Details['event'] = Athlete_Events_Details['event'].str.strip().str.lower()
Athlete_Events_Details['edition'] = Athlete_Events_Details['edition'].str.strip().str.lower()

In [ ]:
Athlete_Events_Details.head(10)

In [ ]:
Athlete_Events_Details['edition'] = Athlete_Events_Details['edition'].astype('category')
Athlete_Events_Details['isTeamSport'] = Athlete_Events_Details['isTeamSport'].astype(bool)

In [ ]:
Athlete_Events_Details.head(10)

In [ ]:
Athlete_Events_Details[['year', 'olympic_type']] = Athlete_Events_Details['edition'].str.extract(r'(\d{4})\s+(summer|winter)\s+olympics')

print(Athlete_Events_Details['year'].head())
print(Athlete_Events_Details['olympic_type'].head())

In [ ]:
Athlete_Events_Details['men'] = Athlete_Events_Details['event'].str.contains(r'\bmen\b', case=False).astype(int)
Athlete_Events_Details['women'] = Athlete_Events_Details['event'].str.contains('women', case=False).astype(int)

Athlete_Events_Details.head(10)

In [ ]:
Athlete_Events_Details.head(10)

# Country_Profile Cleaning
Cleaned by Geetansh

Just doing to lower for all string datatype column.

In [ ]:
Country_Profile['noc'] = Country_Profile['noc'].str.strip().str.lower()
Country_Profile['country'] = Country_Profile['country'].str.strip().str.lower()

In [ ]:
print(Country_Profile.head(10))

# Exploratory Data Analysis (Sataakshi 2)

> Country-wise Comparison of Team vs. Individual Sport Medals Over the Years

In [ ]:
!pip install pandasql


In [ ]:
import pandasql as ps

The SQL query extracts the year-wise medal count data for the USA from the Athlete_Events_Details table. It focuses on both individual and team sports, counting only the events where athletes won medals. Here’s what the query does step by step:

Columns Selected:

year: The specific year of the Olympic Games.
country_noc: The country code, filtered for the USA ('usa').
isTeamSport: Indicates whether the event was a team sport or an individual sport.

medal_count: The count of medals won by USA athletes in that year, for both team and individual sports.
Filtering:Excludes rows where the athlete did not win a medal (medal != 'no medal').Ensures that the year column is not null.

Grouping: Groups the results by year, country_noc, and isTeamSport, to get the total medal count for each year based on whether it was a team or individual sport.

Ordering:The results are ordered by year, country_noc, and isTeamSport to present the data in chronological order, further split into team and individual sport categories.

The queries below generates a summary of the various countries medal wins, categorized by year and sport type, allowing analysis of trends in team vs. individual performance over time.

In [ ]:
query_usa = """
    SELECT
    year,
    country_noc,
    isTeamSport,
    COUNT(medal) AS medal_count
    FROM Athlete_Events_Details
    WHERE medal != 'no medal'
    AND year IS NOT NULL
    AND country_noc IN ('usa')
    GROUP BY year, country_noc, isTeamSport
    ORDER BY year, country_noc, isTeamSport;
    """

medals_yearwise_data_usa = ps.sqldf(query_usa, locals())

print(medals_yearwise_data_usa)


In [ ]:
query_ger = """
    SELECT
    year,
    country_noc,
    isTeamSport,
    COUNT(medal) AS medal_count
    FROM Athlete_Events_Details
    WHERE medal != 'no medal'
    AND year IS NOT NULL
    AND country_noc IN ('ger')
    GROUP BY year, country_noc, isTeamSport
    ORDER BY year, country_noc, isTeamSport;
    """

medals_yearwise_data_ger = ps.sqldf(query_ger, locals())

print(medals_yearwise_data_ger)


In [ ]:
query_ita = """
    SELECT
    year,
    country_noc,
    isTeamSport,
    COUNT(medal) AS medal_count
    FROM Athlete_Events_Details
    WHERE medal != 'no medal'
    AND year IS NOT NULL
    AND country_noc IN ('ita')
    GROUP BY year, country_noc, isTeamSport
    ORDER BY year, country_noc, isTeamSport;
    """

medals_yearwise_data_ita = ps.sqldf(query_ita, locals())

print(medals_yearwise_data_ita)


In [ ]:
query_aus = """
    SELECT
    year,
    country_noc,
    isTeamSport,
    COUNT(medal) AS medal_count
    FROM Athlete_Events_Details
    WHERE medal != 'no medal'
    AND year IS NOT NULL
    AND country_noc IN ('aus')
    GROUP BY year, country_noc, isTeamSport
    ORDER BY year, country_noc, isTeamSport;
    """

medals_yearwise_data_aus = ps.sqldf(query_aus, locals())

print(medals_yearwise_data_aus)


In [ ]:
query_ind = """
    SELECT
    year,
    country_noc,
    isTeamSport,
    COUNT(medal) AS medal_count
    FROM Athlete_Events_Details
    WHERE medal != 'no medal'
    AND year IS NOT NULL
    AND country_noc IN ('ind')
    GROUP BY year, country_noc, isTeamSport
    ORDER BY year, country_noc, isTeamSport;
    """

medals_yearwise_data_ind = ps.sqldf(query_ind, locals())

print(medals_yearwise_data_ind)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

colours = {True: 'red', False: 'blue'}

plt.figure(figsize=(14, 8))
sns.lineplot(
    data=medals_yearwise_data_usa,
    x='year',
    y='medal_count',
    hue='isTeamSport',
    palette=colours,
    markers=True,
    dashes=False
)

plt.title('Year-wise Team vs Individual Sport Medals: USA', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Number of Medals', fontsize=14)
plt.legend(title='Country (Team vs Individual)', loc='upper left', bbox_to_anchor=(1, 1))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


**Interpretation**: The USA has consistently excelled in team sports, with the medal count for team events always surpassing that of individual sports. At no point in time have individual athletes outperformed the team in terms of total medals. This trend is clearly reflected in the graph, where team sports (in red) consistently show a higher medal count compared to individual sports (in blue) across all years.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

colours = {True: 'red', False: 'blue'}

plt.figure(figsize=(14, 8))
sns.lineplot(
    data=medals_yearwise_data_ger,
    x='year',
    y='medal_count',
    hue='isTeamSport',
    palette=colours,
    markers=True,
    dashes=False
)

plt.title('Year-wise Team vs Individual Sport Medals: GER', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Number of Medals', fontsize=14)
plt.legend(title='Country (Team vs Individual)', loc='upper left', bbox_to_anchor=(1, 1))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()



**Interpretation**: Germany has consistently excelled in team sports, with the medal count for team events always surpassing that of individual sports. There has always been a significant gap between the two, indicating that individual athletes have never outperformed the team in terms of total medals. This trend is clearly illustrated in the graph, where team sports (in red) consistently show a higher medal count compared to individual sports (in blue) across all years.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

colours = {True: 'red', False: 'blue'}

plt.figure(figsize=(14, 8))
sns.lineplot(
    data=medals_yearwise_data_ita,
    x='year',
    y='medal_count',
    hue='isTeamSport',
    palette=colours,
    markers=True,
    dashes=False
)

plt.title('Year-wise Team vs Individual Sport Medals: ITA', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Number of Medals', fontsize=14)
plt.legend(title='Country (Team vs Individual)', loc='upper left', bbox_to_anchor=(1, 1))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

colours = {True: 'red', False: 'blue'}

plt.figure(figsize=(14, 8))
sns.lineplot(
    data=medals_yearwise_data_aus,
    x='year',
    y='medal_count',
    hue='isTeamSport',
    palette=colours,
    markers=True,
    dashes=False
)

plt.title('Year-wise Team vs Individual Sport Medals: AUS', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Number of Medals', fontsize=14)
plt.legend(title='Country (Team vs Individual)', loc='upper left', bbox_to_anchor=(1, 1))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()



**Interpretation**: Except a few years Australia has also always been ahead in team sports. Given the potential, Australia can dedicate their energy towards also increasing the individual medal count

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

colours = {True: 'red', False: 'blue'}

plt.figure(figsize=(14, 8))
sns.lineplot(
    data=medals_yearwise_data_ind,
    x='year',
    y='medal_count',
    hue='isTeamSport',
    palette=colours,
    markers=True,
    dashes=False
)

plt.title('Year-wise Team vs Individual Sport Medals: IND', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Number of Medals', fontsize=14)
plt.legend(title='Country (Team vs Individual)', loc='upper left', bbox_to_anchor=(1, 1))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()



**Interpretation**: For India there is a significant gap in the team sports and individual spots performance. This graph can motivate the country to reduce this gap and prepare for the forthecoming olympics.

# Exploratory Data Analysis (Sataakshi 1)

> Sport-wise Trend of Women’s Participation Over the Years

In this analysis, we examine the participation of women in various sports across different Olympic years using the Athlete_Events_Details dataset. The SQL query retrieves the count of female athletes participating in each sport for every Olympic year, focusing on events where women were represented.


In [ ]:
query = """
    SELECT
    sport,
    year,
    COUNT(athlete_id) AS women_participation_count
FROM Athlete_Events_Details
WHERE
    women = 1
    AND year IS NOT NULL
GROUP BY sport, year
ORDER BY year DESC, women_participation_count ;
    """

women_participation = ps.sqldf(query, locals())

print(women_participation)

The data is filtered to include only instances where women participated, ensuring accurate counts for each sport. The results are grouped by sport and year, providing a clear overview of trends in women’s participation over time.

For each selected sport, a line plot visualizes the year-wise participation count of female athletes, allowing for a comparison of trends and growth in women's representation across different sports. This analysis helps to highlight advancements in gender equality in sports and can identify specific sports where women’s participation has significantly increased or decreased over the years.

The following visualization focuses on women's participation in wrestling, showcasing how participation has evolved through the years.

In [ ]:
selected_sport = 'wrestling'
filtered_sport_data = women_participation[women_participation['sport'] == selected_sport]

plt.figure(figsize=(5, 4))
sns.lineplot(
    data=filtered_sport_data,
    x='year',
    y='women_participation_count',
    marker='o',
    dashes=False
)

plt.title(f'Year-wise Women Participation in {selected_sport}', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Women Participation Count', fontsize=14)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


The following visualization focuses on women's participation in alpine skiing, showcasing how participation has evolved through the years.

In [ ]:
selected_sport = 'alpine skiing'
filtered_sport_data = women_participation[women_participation['sport'] == selected_sport]

plt.figure(figsize=(10, 6))
sns.lineplot(
    data=filtered_sport_data,
    x='year',
    y='women_participation_count',
    marker='o',
    dashes=False
)

plt.title(f'Year-wise Women Participation in {selected_sport}', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Women Participation Count', fontsize=14)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


The following visualization focuses on women's participation in badminton, showcasing how participation has evolved through the years.

In [ ]:

selected_sport = 'badminton'
filtered_sport_data = women_participation[women_participation['sport'] == selected_sport]

plt.figure(figsize=(10, 6))
sns.lineplot(
    data=filtered_sport_data,
    x='year',
    y='women_participation_count',
    marker='o',
    dashes=False
)

plt.title(f'Year-wise Women Participation in {selected_sport}', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Women Participation Count', fontsize=14)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


The following visualization focuses on women's participation in athletics, showcasing how participation has evolved through the years.

In [ ]:
selected_sport = 'athletics'
filtered_sport_data = women_participation[women_participation['sport'] == selected_sport]

plt.figure(figsize=(10, 6))
sns.lineplot(
    data=filtered_sport_data,
    x='year',
    y='women_participation_count',
    marker='o',
    dashes=False
)

plt.title(f'Year-wise Women Participation in {selected_sport}', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Women Participation Count', fontsize=14)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


# ML Model (Sataakshi 2)

> Country-wise Comparison of Team vs. Individual Sport Medals Over the Years


1.   Split dataset into training 80% and testing 20%

2.   Max depth set to 5 to avoid overfitting

3. Using plot tree to visualize decision tree.

4. R² score measures the variance in the data

5. Mean Squared Error masures the average squared difference between the predicted and actual values.



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

medals_yearwise_data_aus['isTeamSport'] = medals_yearwise_data_aus['isTeamSport'].apply(lambda x: 1 if x else 0)

X = medals_yearwise_data_aus[['year', 'isTeamSport']]
y = medals_yearwise_data_aus['medal_count']

# Split dataset into training 80% and testing 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Max depth set to 5 to avoid overfitting
reg_tree = DecisionTreeRegressor(max_depth=5, random_state=42)

reg_tree.fit(X_train, y_train)

plt.figure(figsize=(10, 8))

# Using plot tree to visualize decision tree
plot_tree(reg_tree, feature_names=['year', 'isTeamSport'], filled=True)
plt.show()

y_pred_tree = reg_tree.predict(X_test)
print("Decision Tree-")

# R² score measures the variance in the data
print(f'R² score: {r2_score(y_test, y_pred_tree)}')
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred_tree)}')
print("-------------")

reg_random_forest = RandomForestRegressor(n_estimators=100, random_state=42)

reg_random_forest.fit(X_train, y_train)

y_pred_rf = reg_random_forest.predict(X_test)
print("Random Forest Results- ")

# Mean Squared Error masures the average squared difference between the predicted and actual values.
print(f'R² score: {r2_score(y_test, y_pred_rf)}')
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred_rf)}')



The output shows the performance of two regression models—Decision Tree and Random Forest—in predicting Australia's medal counts.

Decision Tree:

R² score: 0.36, indicating that the model explains about 36% of the variance in the data.

Mean Squared Error (MSE): 834.71, which represents the average squared difference between actual and predicted medal counts.

Random Forest:

R² score: 0.51, meaning it explains 51% of the variance, making it a better model compared to the Decision Tree.

MSE: 641.43, which is lower than the Decision Tree, indicating better prediction accuracy.

Overall, the Random Forest performs better than the Decision Tree, offering higher accuracy and better variance explanation.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

medals_yearwise_data_ind['isTeamSport'] = medals_yearwise_data_ind['isTeamSport'].apply(lambda x: 1 if x else 0)

X = medals_yearwise_data_ind[['year', 'isTeamSport']]
y = medals_yearwise_data_ind['medal_count']

# Split dataset into training 80% and testing 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Max depth set to 5 to avoid overfitting
reg_tree = DecisionTreeRegressor(max_depth=5, random_state=42)

reg_tree.fit(X_train, y_train)

plt.figure(figsize=(10, 8))

# Using plot tree to visualize decision tree
plot_tree(reg_tree, feature_names=['year', 'isTeamSport'], filled=True)
plt.show()

y_pred_tree = reg_tree.predict(X_test)
print("Decision Tree-")

# R² score measures the variance in the data
print(f'R² score: {r2_score(y_test, y_pred_tree)}')
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred_tree)}')
print("-------------")

reg_random_forest = RandomForestRegressor(n_estimators=100, random_state=42)

reg_random_forest.fit(X_train, y_train)

y_pred_rf = reg_random_forest.predict(X_test)
print("Random Forest Results- ")

# Mean Squared Error masures the average squared difference between the predicted and actual values.
print(f'R² score: {r2_score(y_test, y_pred_rf)}')
print(f'Mean Squared Error: {mean_squared_error(y_test, y_pred_rf)}')



The output for India indicates that both models-Decision Tree and Random Forest perform well.

Decision Tree:-

R² score: 0.88 meaning 88% of the variance in the data is explained by the model.

Mean Squared Error: 8.4 shows the average squared difference between the predicted and actual values.

Random Forest:-

R² score: 0.92 means that there is a slight improvement, meaning the model explains 92% of the variance.

Mean Squared Error: The lower MSE value of 5.7 shows that Random Forest makes more accurate predictions than the Decision Tree in this case.

In summary, both models perform well, with Random Forest slightly outperforming the Decision Tree in predictive accuracy.

# ML Model (Sataakshi 1)

> Sport-wise Trend of Women’s Participation Over the Years

In [ ]:
query = """
    SELECT
        sport,
        year,
        COUNT(athlete_id) AS women_participation_count
    FROM Athlete_Events_Details
    WHERE
        women = 1
        AND year IS NOT NULL
    GROUP BY sport, year
    ORDER BY sport, year ASC;
"""

women_participation = ps.sqldf(query, locals())

print(women_participation.head())
plt.figure(figsize=(12, 8))

sns.lineplot(data=women_participation, x='year', y='women_participation_count', hue='sport', marker='o', dashes=False)

plt.title('Year-wise Women Participation in All Sports', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Women Participation Count', fontsize=14)
plt.xticks(rotation=45)
plt.legend(title='Sport', bbox_to_anchor=(1.05, 1), loc='upper left')

plt.tight_layout()
plt.show()


In [ ]:
query = """
    SELECT
        sport,
        COUNT(DISTINCT year) AS years_practiced
    FROM Athlete_Events_Details
    WHERE
        women = 1
        AND year IS NOT NULL
    GROUP BY sport
    ORDER BY years_practiced DESC
    LIMIT 5;  -- Or adjust to any number
"""

topSports = ps.sqldf(query, locals())

print(topSports)


We have taken a list of sports which had maximum data over the course of olympic history.
Such sports include swimming, diving, fencing, athletics. These four sports are known to be some of the oldest sports in the olympic history.

Next, we are going to iterate over every sport and filter the required data. Using the KNN model we will split the data and use the Knn regression to calculate the preedicted value.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# List of sports
sports = [ 'swimming', 'diving', 'fencing', 'athletics']

rmseVal = {}
r2Val = {}

# Loop over each sport to train and evaluate the k-NN model.
for idx, sport in enumerate(sports):

    # Filter data for the current sport.
    participationData = women_participation[women_participation['sport'] == sport]

    X = participationData[['year']]
    y = participationData['women_participation_count']

    XTrain, XTest, yTrain, yTest = train_test_split(X, y, test_size=0.3, random_state=42)

    scaler = StandardScaler()
    XTrainScaled = scaler.fit_transform(XTrain)
    XTestScaled = scaler.transform(XTest)

    knnReg = KNeighborsRegressor(min(5, len(XTrain)))
    knnReg.fit(XTrainScaled, yTrain)

    yPredKnn = knnReg.predict(XTestScaled)

    # Calculate RMSE and R² score
    rmseKnn = np.sqrt(mean_squared_error(yTest, yPredKnn))
    r2Knn = r2_score(yTest, yPredKnn)

    rmseVal[sport] = rmseKnn
    r2Val[sport] = r2Knn

    print(f"{sport.capitalize()} Results")
    print(f'R² score: {r2Knn:.2f}')
    print(f'Root Mean Square Error: {rmseKnn:.2f}')

    sortedVal = np.argsort(X_test.values.flatten())
    XTestSorted = X_test.values.flatten()[sortedVal]
    yTestSorted = y_test.values.flatten()[sortedVal]
    yPredKnnSorted = yPredKnn[sortedVal]

    plt.figure(figsize=(8, 5))

    # Plot actual vs predicted values for the current sport
    plt.plot(XTestSorted, yTestSorted, color='blue', label=f'Actual {sport.capitalize()}')
    plt.plot(XTestSorted, yPredKnnSorted, color='red', linestyle='dashed', label=f'Predicted {sport.capitalize()}')

    plt.title(f"k-NN Regression - Women Participation in {sport.capitalize()}", fontsize=14)
    plt.xlabel("Year")
    plt.ylabel("Women's Participation Count")
    plt.legend(loc='upper left', fontsize=10)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

print("\nOverall Results:")
for sport in sports:
    if sport in rmseVal:
        print(f"{sport.capitalize()} -> RMSE: {rmseVal[sport]:.2f}, R²: {r2Val[sport]:.2f}")


The output indicates the performance of the k-NN regression model for predicting women's participation in four sports:

Results per Sport:

Swimming:-

R²: 0.73 - The model explains 73% of the variance in swimming participation.

RMSE: 125.06 - The model's predictions are off by an average of 125.06 units.

Diving:-

R²: 0.77 - The model explains 77% of the variance in diving participation.

RMSE: 10.88 - The predictions are off by an average of 10.88 units.

Fencing:-

R²: 0.59 - The model explains 59% of the variance in fencing participation.

RMSE: 32.65 - The predictions are off by an average of 32.65 units.

Athletics:-

R²: 0.89 - The model explains 89% of the variance in athletics participation.

RMSE: 131.02 - The predictions are off by an average of 131.02 units.

Overall Summary:

Swimming and Athletics show strong model performance. R² values of 0.73 and 0.89. However, Athletics has a higher RMSE, indicating larger prediction errors compared to Diving and Fencing.
Diving has the best balance of a relatively high R² (0.77) and a low RMSE (10.88), suggesting the model makes accurate predictions for diving.
Fencing has the lowest R² (0.59) and a moderate RMSE (32.65), indicating room for improvement in the model's performance.
The plots show how well the k-NN model captures the trend of participation for each sport over time. The dashed red line represents the predicted values, while the blue line represents the actual participation data.